In [1]:
import psycopg2
import pandas as pd
import datetime
from psycopg2 import OperationalError
import openpyxl 

In [2]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    """Функция для подключения к базе данных PostgreSQL"""
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        connection.autocommit = True 
        print("Подключение к БД успешно")
    except OperationalError as e:
        print(f"Ошибка подключения: {e}")
    return connection

In [3]:
def create_query(query:str, connection, select=False):
    """Функция для отправки запроса к базе данных"""
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        if select:
            data = cursor.fetchall()
            cursor.close()
            return data
            
        cursor.close()
    except OperationalError as e:
        print(f"Ошибка запроса: {e}")   

### Создание базы данных и таблиц

In [4]:
# Создание базы данных
connection = psycopg2.connect(user="postgres",
                              password="111",
                              host="127.0.0.1",
                              port="5432")

connection.autocommit = True 
create_query("CREATE DATABASE info_vizion", connection)    
connection.close()    

In [5]:
# Создание таблиц
connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

# Календаря
query_calendar = """CREATE TABLE calendar (
                                            date_key DATE PRIMARY KEY
                                           )"""
create_query(query_calendar, connection)

# Складов
query_stocks = """CREATE TABLE stocks (
                                        id INTEGER PRIMARY KEY,
                                        stock_name VARCHAR(15) NOT NULL
                                       )"""
create_query(query_stocks, connection)

# Транзакций
query_sales = """CREATE TABLE sales (
                                      id SERIAL PRIMARY KEY,
                                      stock_sale INTEGER NOT NULL,
                                      date_sale DATE NOT NULL,
                                      sum_sale NUMERIC NOT NULL,
                                      CONSTRAINT stock_sale FOREIGN KEY (stock_sale)
                                          REFERENCES stocks (id) MATCH SIMPLE
                                    )"""
create_query(query_sales, connection)

connection.close()    

Подключение к БД успешно


### Заполнение таблиц

**Календарь**

In [6]:
date_min = datetime.date(2012, 1, 1) 
date_max = datetime.date(2015, 12, 31)

connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

while date_min<=date_max:
    query_date = f"INSERT INTO calendar (date_key) VALUES ('{date_min.strftime('%d.%m.%Y')}')"
    create_query(query_date, connection)
    date_min += datetime.timedelta(days=1) 

connection.close()

Подключение к БД успешно


In [7]:
df = pd.read_csv('ИсходныеДанныеИПримерыОтчетов.csv', sep=';')
df.head(3)

,Склад,ДатаКлюч,Продажи рублей
0,Склад 1,08.01.2013,1776309.0
1,Склад 1,09.01.2013,1670458.8
2,Склад 1,13.01.2013,2065636.4


**Склады**

In [8]:
stock_list = list(df['Склад'].unique())
stocks = [(int(el.replace('Склад ', '')), el) for el in stock_list]
stocks

[(1, 'Склад 1'),
 (143, 'Склад 143'),
 (206, 'Склад 206'),
 (227, 'Склад 227'),
 (23, 'Склад 23'),
 (51, 'Склад 51'),
 (66, 'Склад 66')]

In [9]:
connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

for el in stocks:
    query_stock = f"INSERT INTO stocks (id, stock_name) VALUES {el}"
    create_query(query_stock, connection)
      
connection.close()

Подключение к БД успешно


**Транзакции**

In [10]:
connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

for i in range(df.shape[0]):
    el_insert = (int(df['Склад'].iloc[i].replace('Склад ', '')), df['ДатаКлюч'].iloc[i], df['Продажи рублей'].iloc[i])
    query_sale = f"INSERT INTO sales (stock_sale, date_sale, sum_sale ) VALUES {el_insert}"
    create_query(query_sale, connection)
      
connection.close()

Подключение к БД успешно


### Запросы на создание таблиц

In [11]:
query_select_day = """
SELECT 
st.stock_name "Склад",  
cln.date_key "Дата", 
sl.sum_sale/1000 "Продажи, тыс.руб.",

sl_Y.date_sale "Прошлый год",
sl_Y.sum_sale/1000 "Продажи за аналогичный период предыдущего года, тыс.руб.",

sl_M.date_sale "Прошлый месяц",
sl_M.sum_sale/1000 "Продажи за аналогичный период предыдущего месяца, тыс. руб."


FROM sales sl
FULL JOIN sales sl_Y ON sl.date_sale = sl_Y.date_sale +  interval '1 year' AND sl_Y.stock_sale = sl.stock_sale
FULL JOIN sales sl_M ON sl.date_sale = sl_M.date_sale +  interval '1 month' AND sl_M.stock_sale = sl.stock_sale
LEFT JOIN stocks st ON st.id = sl.stock_sale OR st.id = sl_Y.stock_sale OR st.id = sl_M.stock_sale
LEFT JOIN calendar cln ON cln.date_key = sl.date_sale 
                       OR cln.date_key=sl_Y.date_sale + interval '1 year' 
                       OR cln.date_key=sl_M.date_sale + interval '1 month'
WHERE cln.date_key BETWEEN '2013-01-01' AND '2015-12-31'
Order by st.id, cln.date_key
"""

In [12]:
connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

data_day = create_query(query_select_day, connection, select=True)
      
connection.close()

Подключение к БД успешно


In [14]:
df_day = pd.DataFrame(data_day, columns=['Склад', "Дата", "Продажи, тыс.руб.", "Прошлый год", 
                            "Продажи за аналогичный период предыдущего года, тыс.руб.",
                           "Прошлый месяц", "Продажи за аналогичный период предыдущего месяца, тыс. руб."])
df_day.sample(3)

,Склад,Дата,"Продажи, тыс.руб.",Прошлый год,"Продажи за аналогичный период предыдущего года, тыс.руб.",Прошлый месяц,"Продажи за аналогичный период предыдущего месяца, тыс. руб."
751,Склад 23,2013-09-03,972.7664000000000000,None,None,None,None
3123,Склад 143,2015-10-04,None,None,None,2015-09-04,999.1910000000000000
2117,Склад 66,2014-02-27,None,None,None,2014-01-27,1161.7446000000000000


In [15]:
query_select_month = """
SELECT 
st.stock_name "Склад", 

CONCAT(
EXTRACT(YEAR FROM cln.date_key), '.',
EXTRACT(MONTH FROM cln.date_key) 
) "Год-Месяц",

SUM(sl.sum_sale/1000) "Продажи_тыс_руб",
SUM(sl_Y.sum_sale/1000) "Продажи_за_аналогичный_период_пред_года_тыс_руб",
SUM(sl_M.sum_sale/1000) "Продажи_за_аналогичный_период_пред_месяца_тыс_руб"


FROM sales sl
FULL JOIN sales sl_Y ON sl.date_sale = sl_Y.date_sale +  interval '1 year' AND sl_Y.stock_sale = sl.stock_sale
FULL JOIN sales sl_M ON sl.date_sale = sl_M.date_sale +  interval '1 month' AND sl_M.stock_sale = sl.stock_sale
LEFT JOIN stocks st ON st.id = sl.stock_sale OR st.id = sl_Y.stock_sale OR st.id = sl_M.stock_sale
LEFT JOIN calendar cln ON cln.date_key = sl.date_sale 
                       OR cln.date_key=sl_Y.date_sale + interval '1 year' 
                       OR cln.date_key=sl_M.date_sale + interval '1 month'
WHERE cln.date_key BETWEEN '2013-01-01' AND '2015-12-31'
GROUP BY st.stock_name, EXTRACT(YEAR FROM cln.date_key), EXTRACT(MONTH FROM cln.date_key)
Order by st.stock_name, EXTRACT(YEAR FROM cln.date_key), EXTRACT(MONTH FROM cln.date_key)

"""

In [16]:
connection = create_connection("info_vizion", "postgres", "111", "127.0.0.1", "5432")

data_mount = create_query(query_select_month, connection, select=True)
      
connection.close()

Подключение к БД успешно


In [17]:
df_month = pd.DataFrame(data_mount, columns=['Склад', "Год-Месяц", "Продажи_тыс_руб", 
                            "Продажи_за_аналогичный_период_пред_года_тыс_руб",
                            "Продажи_за_аналогичный_период_пред_месяца_тыс_руб"])
df_month.sample(3)

,Склад,Год-Месяц,Продажи_тыс_руб,Продажи_за_аналогичный_период_пред_года_тыс_руб,Продажи_за_аналогичный_период_пред_месяца_тыс_руб
172,Склад 23,2015.5,14765.2301000000000000,12545.4093000000000000,8725.2173000000000000
11,Склад 1,2013.12,7575.7882000000000000,None,8813.7274999999980000
116,Склад 227,2013.9,4690.5801999999990000,None,5225.8761999999950000


In [18]:
with pd.ExcelWriter('отчеты.xlsx') as writer:  
    df_day.to_excel(writer, sheet_name='По дням', index=False)
    df_month.to_excel(writer, sheet_name='По месяцам', index=False)


**Запрос по дням**
![image info](./img/Запрос_по_дням.png)

**Запрос по месяцам**
![image info](./img/Запрос_по_месяцам.png)

In [ ]:
# # Удаление базы данных
# connection = psycopg2.connect(user="postgres",
#                               password="111",
#                               host="127.0.0.1",
#                               port="5432")
# connection.autocommit = True  
# create_query("DROP DATABASE info_vizion", connection)    
# connection.close()   